# 0. Setup
Please make sure your environment is set up according to the instructions here: https://github.com/NASA-NAVO/aas_workshop_2020_winter/blob/master/00_SETUP.md

Ensure you have the latest version of the workshop material by updating your environment:
TBD

# 1. Overview
NASA services can be queried from Python in multiple ways.
* Generic Virtual Observatory (VO) queries.
  * Call sequence is consistent, including for non-NASA resources.
  * Use the `pyvo` package: https://pyvo.readthedocs.io/en/latest/
  * Known issues/caveats: https://github.com/NASA-NAVO/aas_workshop_2020_winter/blob/master/KNOWN_ISSUES.md
* Astroquery interfaces
  * Call sequences not quite as consistent, but follow similar patterns.
  * See https://astroquery.readthedocs.io/en/latest/
  * Informal Q&A session Tuesday, 5:30pm-6:30pm, NumFocus booth
* Ad hoc archive-specific interfaces

# 2. VO Services
This workshop will introduce 4 types of VO queries:
* **VO Registry** - Discover what services are available worldwide
* **Simple Cone Search** - Search for catalog object within a specified cone region
* **Simple Image Access** - Search for image products within a spatial region
* **Simple Spectral Access** - Search for spectral products within a spatial region
* **Table Access** - SQL-like queries to databases

## 2.1 Import Necessary Packages

In [1]:
# Generic VO access routines
import pyvo as vo

# For specifying coordinates and angles
from astropy.coordinates import SkyCoord
from astropy.coordinates import Angle
from astropy import units as u

# For downloading files
from astropy.utils.data import download_file

# Ignore unimportant warnings
import warnings
warnings.filterwarnings('ignore', '.*Unknown element mirrorURL.*', vo.utils.xml.elements.UnknownElementWarning)

## 2.1 Look Up Services in VO Registry
Simple example:  Find Simple Cone Search (conesearch) services related to SWIFT.

In [2]:
services = vo.regsearch(servicetype='conesearch', keywords=['swift'])
services

<Table length=185>
                 ivoid                  ...
                                        ...
                 object                 ...
--------------------------------------- ...
ivo://archive.stsci.edu/catalogs/mastcs ...
    ivo://bsdc.icranet.org/sds82/q/cone ...
               ivo://cds.vizier/b/swift ...
                ivo://cds.vizier/ii/339 ...
                ivo://cds.vizier/ii/339 ...
                 ivo://cds.vizier/ix/43 ...
                 ivo://cds.vizier/ix/43 ...
                 ivo://cds.vizier/ix/58 ...
                 ivo://cds.vizier/ix/58 ...
                                    ... ...
          ivo://nasa.heasarc/swiftmastr ...
          ivo://nasa.heasarc/swifttdrss ...
          ivo://nasa.heasarc/swiftuvlog ...
          ivo://nasa.heasarc/swiftxrlog ...
           ivo://nasa.heasarc/swsdssqso ...
           ivo://nasa.heasarc/swuvotssc ...
          ivo://nasa.heasarc/swuvotssob ...
            ivo://nasa.heasarc/swxcscat ...
           iv

### 2.1.1 Use different arguments/values to modify the simple example
| Argument | Description | Examples |
| :-----: | :----------- | :-------- |
| **servicetype** | Type of service | `conesearch` or `scs` for **Simple Cone Search**<br> `image` or `sia` for **Simple Image Access**<br> `spectrum` or `ssa` for **Simple Spectral Access**<br> `table` or `tap` for **Table Access Protocol**|
| **keyword** | List of one or more keyword(s) to match service's metadata. Both ORs and ANDs may be specified.<br><ul><li>(OR) A list of keywords match a service if **any** of the keywords match the service.</li><li>(AND) If a  keyword contains multiple space-delimited words, **all** the words must match the metadata.</li></ul>| `['galex', 'swift']` matches 'galex' or 'swift'<br>`['hst survey']` matches services mentioning both 'hst' and 'survey' |
| **waveband** | Resulting services have data in the specified waveband(s) | ‘radio’, ‘millimeter’, ‘infrared’, ‘optical’, ‘uv’, ‘euv’, ‘x-ray’ ‘gamma-ray’ |

### 2.1.2 Inspect the results.
#### Using pyvo
Although not lists, `pyvo` results can be iterated over to see each individual result.  The results are specialized based on the type of query, providing access to the important properties of the results.  Some useful accessors with registry results are:
* `short_name` - A short name
* `res_title` - A more descriptive title
* `res_description` - A more verbose description
* `reference_url` - A link for more information
* `ivoid` - A unique identifier for the service.  Gives some indication of what organization is serving the data.

In [3]:
# Print the number of results and the 1st 4 short names and titles.
print(f'Number of results: {len(services)}\n')
for s in list(services)[:4]:  # (Treat services as list to get the subset of rows)
    print(f'{s.short_name} - {s.res_title}')

Number of results: 185

MAST CS - MAST ConeSearch
sds82 cone - SDS82 catalog
B/swift - Swift Master Catalog (HEASARC, 2004-)
II/339 - Swift/UVOT Serendipitous Source Catalog (Yershov, 2015)


#### Filtering results
Of the services we found, which one(s) have 'stsci.edu' in their unique identifier?

In [4]:
stsci_services = [s for s in services if 'stsci.edu' in s.ivoid]
for s in stsci_services:
    print (f'(STScI): {s.short_name} - {s.res_title}')

(STScI): MAST CS - MAST ConeSearch


#### Using astropy
With the `to_table()` method, `pyvo` results can also be converted to Astropy `Table` objects which offer a variety of addional features. See http://docs.astropy.org/en/stable/table/ for more on working with Astropy Tables.

In [5]:
# Convert to an Astropy Table
services_table = services.to_table()

# Print the column names and display 1st 3 rows with a subset of columns
print(f'\nColumn Names:\n{services_table.colnames}\n')  
services_table['short_name', 'res_title', 'res_description'][:3]  


Column Names:
['ivoid', 'cap_index', 'intf_index', 'intf_type', 'intf_role', 'std_version', 'query_type', 'result_type', 'wsdl_url', 'url_use', 'access_url', 'mirror_url', 'authenticated_only', 'security_method_id', 'ivoid_', 'cap_index_', 'cap_type', 'cap_description', 'standard_id', 'ivoid__', 'res_type', 'created', 'short_name', 'res_title', 'updated', 'content_level', 'res_description', 'reference_url', 'creator_seq', 'content_type', 'source_format', 'source_value', 'res_version', 'region_of_regard', 'waveband', 'rights', 'rights_uri', 'harvested_from']



short_name,res_title,res_description
object,object,object
MAST CS,MAST ConeSearch,"All MAST catalog holdings are available via a ConeSearch endpoint. This service provides access to all, with an optional non-standard parameter for an individual catalog to query. The available missions are listed at http://archive.stsci.edu/vo/mast_services.html, and include Hubble (HST) data, Kepler, K2, IUE, HUT, EUVE, FUSE, UIT, WUPPE, BEFS, TUES, IMAPS, High Level Science Products (HLSP), Copernicus, HPOL, VLA First, XMM-OM, and SWIFT."
sds82 cone,SDS82 catalog,The SDS82 catalog is the result of the Swift DeepSky pipeline over all observations made by Swift-XRT within the Stripe82.
B/swift,"Swift Master Catalog (HEASARC, 2004-)","This table records high-level information for each Swift observation and provides access to the data archive. Each record is associated with a single observation that contains data from all instruments on board Swift. The BAT is the large field of view instrument and operates in the 10-300 keV energy band. The narrow field instruments, XRT and UVOT, operate in the X-ray and UV/optical regime, respectively. An observation is defined as a collection of snapshots, where a snapshot is defined as the time spent observing the same position continuously. Because of observing constraints, the length of a snapshot can be shorter than a single orbit and it can be interrupted because the satellite will point in a different direction of the sky or because the time allocated to that observation ends. The typical Swift observing strategy for a Gamma Ray Burst (GRB) and/or afterglow, consists of a serious of observations aimed at following the GRB and its afterglow evolution. This strategy is achieved with two different type of observations named Automatic Targets and Pre-Planned Targets. The Automatic Target is initiated on board soon after an event is triggered by the BAT. The Figure of Merit (FOM) algorithm, part of the observatory's autonomy, decides if it is worth requesting a slew maneuver to point the narrow field instruments (NFI) on Swift, XRT and UVOT, in the direction of the trigger. If the conditions to slew to the new position are satisfied, the Automatic Target observation takes place; all the instruments have a pre-set standard configuration of operating modes and filters and about 20000 seconds on source will be collected. The Pre-Planned Target observations instead are initiated from the ground once the trigger is known. These observations are planned on ground and uploaded onto the spacecraft."


## 2.2 Cone search
Example:  Find a cone search service for the USNO-B catalog and search it around M51 with a .1 degree radius.  (More inspection could be done on the service list instead of blindly choosing the first service.)  

The position (`pos`) is best specified with `SkyCoord` objects (see http://docs.astropy.org/en/stable/api/astropy.coordinates.SkyCoord.html). 

The size of the region is specified with the `radius` keyword and may be decimal degrees or an Astropy `Angle` (http://docs.astropy.org/en/stable/api/astropy.coordinates.Angle.html#astropy.coordinates.Angle). 

In [6]:
m51_pos = SkyCoord.from_name("m51")
services = vo.regsearch(servicetype='conesearch', keywords='usno-b')
results = services[0].search(pos=m51_pos, radius=0.1)
# Astropy Table is useful for displaying cone search results.
results.to_table()  

ObjID,Zone,SeqNo,RA,DEC,pmRA,pmDEC,e_pmRA,e_pmDEC,e_RA,e_DEC,Epoch,B1Mag,R1s_g,B2Mag,B2s_g,R2Mag,R2s_g,NMag,mag,B1s_g,R1Mag,distance
,,,deg,deg,mas / yr,mas / yr,mas / yr,mas / yr,arcsec,arcsec,yr,mag,,mag,,mag,,mag,mag,,mag,arcsec
int64,int32,int32,float64,float64,float32,float32,float32,float32,float32,float32,float32,float32,int32,float32,int32,float32,int32,float32,float32,int32,float32,float32
5888400445251,1371,282435,202.472919444444,47.1954444444444,2.081668e-13,2.081668e-13,0.0,0.0,999.0,999.0,1981.8,-999999500.0,1,9.4,9,-999999500.0,0,7.86,-999999500.0,0,8.98,0.13680966
5892695420475,1372,290363,202.494625,47.2065472222222,2.081668e-13,2.081668e-13,0.0,0.0,332.0,999.0,1956.2,20.13,3,-999999500.0,0,-999999500.0,0,-999999500.0,-999999500.0,4,15.79,1.2253959
5892695420458,1372,290346,202.436875,47.2094416666667,2.081668e-13,2.081668e-13,0.0,0.0,999.0,359.0,1976.2,-999999500.0,1,-999999500.0,0,-999999500.0,0,10.89,-999999500.0,0,13.27,1.5814877
5892695420468,1372,290356,202.469830555556,47.2232805555556,2.081668e-13,2.081668e-13,0.0,0.0,560.0,934.0,1976.2,-999999500.0,3,-999999500.0,0,-999999500.0,0,18.4,-999999500.0,0,17.78,1.6813658
5888400445256,1371,282440,202.480638888889,47.1682194444444,-20.0,-6.0,5.0,5.0,205.0,201.0,1969.5,12.79,8,-999999500.0,0,-999999500.0,0,14.87,-999999500.0,1,15.4,1.6839056
5892695420469,1372,290357,202.4706,47.2235555555556,2.081668e-13,2.081668e-13,0.0,0.0,709.0,130.0,1956.2,15.89,3,-999999500.0,0,-999999500.0,0,-999999500.0,-999999500.0,1,17.68,1.6983474
5892695420470,1372,290358,202.474338888889,47.2264416666667,2.081668e-13,2.081668e-13,0.0,0.0,200.0,490.0,1956.2,13.49,4,-999999500.0,0,-999999500.0,0,-999999500.0,-999999500.0,1,18.74,1.8810483
5888400445204,1371,282388,202.423558333333,47.1977666666667,2.081668e-13,2.081668e-13,0.0,0.0,466.0,480.0,1969.5,15.66,8,-999999500.0,0,-999999500.0,0,14.97,-999999500.0,1,15.22,1.8820877


## 2.3 Image search
Example:  Find an image search service for GALEX, and search it around coordinates 13:37:00.950,-29:51:55.51 (M83) with a radius of .2 degrees.  Download the first file in the results.
#### Find an image service

In [7]:
services = vo.regsearch(servicetype='image', keywords=['galex'])
services.to_table()['ivoid', 'short_name', 'res_title']

ivoid,short_name,res_title
object,object,object
ivo://archive.stsci.edu/sia/galex,GALEX,Galaxy Evolution Explorer
ivo://irsa.ipac/spitzer/images/lvl,LVL,Spitzer Local Volume Legacy Survey
ivo://irsa.ipac/wise/images/z0mgs,z0MGS,The z=0 Multiwavelength Galaxy Synthesis
ivo://mast.stsci/siap/galex_atlas,GALEX_Atlas,GALEX Atlas of Nearby Galaxies
ivo://nasa.heasarc/skyview/galex,GALEX,Galaxy Explorer All Sky Survey: Near UV


#### Search one of the services
The first service looks good.  Search it!

For more details on using `SkyCoord` see http://docs.astropy.org/en/stable/api/astropy.coordinates.SkyCoord.html#astropy.coordinates.SkyCoord

**NOTE**:  For image searches, the size of the region is defined by the `size` keyword which is more like a diameter than a radius.

In [8]:
m83_pos = SkyCoord('13h37m00.950s -29d51m55.51s')
results = services[0].search(pos=m83_pos, size=.2)

# We can look at the results.
results.to_table()

collection,insName,name,posLocationRA,posLocationDec,contentLength,naxes,naxis,scale,contentType,coordFrame,projection,crpix,crval,cdmatrix,enrEMBand,enrUnits,enrValue,enrMax,enrMin,accessURL
object,object,object,float64,float64,int32,int32,object,object,object,object,str3,object,object,object,object,object,float64,float64,float64,object
GALEX,GALEX,nga_m83_582_f5_19_158-nd-int.fits.gz,203.769181415483,-30.085639855328,16778475,2,[3840 3840],[0.000416667 0.000416667],image/fits,ICRS,TAN,[1920.5 1920.5],[16.4257 -33.5407],[-0.000416667 -0.0 -0.0 0.000416667],UV,meters,2.35e-07,3.007e-07,1.693e-07,http://galex.stsci.edu/data/GR7/pipe/01-vsn/02439-NGA_M83_582_F5_19_158/d/01-main/0007-img/07-try/NGA_M83_582_F5_19_158-nd-int.fits.gz
GALEX,GALEX,nga_m83_582_f5_19_158-xd-int_2color.jpg,203.769181415483,-30.085639855328,2614899,2,[3840 3840],[0.000416667 0.000416667],image/jpeg,ICRS,TAN,[1920.5 1920.5],[38.6497 -6.98734],[-0.000416667 -0.0 -0.0 0.000416667],UV,meters,2.35e-07,3.007e-07,1.693e-07,http://galex.stsci.edu/data/GR7/pipe/01-vsn/02439-NGA_M83_582_F5_19_158/d/01-main/0007-img/07-try/qa/NGA_M83_582_F5_19_158-xd-int_2color.jpg
GALEX,GALEX,nga_m83_582_f5_19_158-xd-int_2color_large.jpg,203.769181415483,-30.085639855328,487683,2,[3840 3840],[0.000416667 0.000416667],image/jpeg,ICRS,TAN,[1920.5 1920.5],[38.6497 -6.98734],[-0.000416667 -0.0 -0.0 0.000416667],UV,meters,2.35e-07,3.007e-07,1.693e-07,http://galex.stsci.edu/data/GR7/pipe/01-vsn/02439-NGA_M83_582_F5_19_158/d/01-main/0007-img/07-try/qa/NGA_M83_582_F5_19_158-xd-int_2color_large.jpg
GALEX,GALEX,nga_m83_582_f5_19_158-xd-int_2color_medium.jpg,203.769181415483,-30.085639855328,531613,2,[3840 3840],[0.000416667 0.000416667],image/jpeg,ICRS,TAN,[1920.5 1920.5],[38.6497 -6.98734],[-0.000416667 -0.0 -0.0 0.000416667],UV,meters,2.35e-07,3.007e-07,1.693e-07,http://galex.stsci.edu/data/GR7/pipe/01-vsn/02439-NGA_M83_582_F5_19_158/d/01-main/0007-img/07-try/qa/NGA_M83_582_F5_19_158-xd-int_2color_medium.jpg
GALEX,GALEX,nga_m83_582_f5_19_158-xd-int_2color_small.jpg,203.769181415483,-30.085639855328,135907,2,[3840 3840],[0.000416667 0.000416667],image/jpeg,ICRS,TAN,[1920.5 1920.5],[18.7794 -32.251],[-0.000416667 -0.0 -0.0 0.000416667],UV,meters,2.35e-07,3.007e-07,1.693e-07,http://galex.stsci.edu/data/GR7/pipe/01-vsn/02439-NGA_M83_582_F5_19_158/d/01-main/0007-img/07-try/qa/NGA_M83_582_F5_19_158-xd-int_2color_small.jpg
GALEX,GALEX,nga_m83_582_f5_19_158-xd-int_2color_medium_annot.jpg,203.769181415483,-30.085639855328,593089,2,[3840 3840],[0.000416667 0.000416667],image/jpeg,ICRS,TAN,[1920.5 1920.5],[49.129 -7.82281],[-0.000416667 -0.0 -0.0 0.000416667],UV,meters,2.35e-07,3.007e-07,1.693e-07,http://galex.stsci.edu/data/GR7/pipe/01-vsn/02439-NGA_M83_582_F5_19_158/d/01-main/0007-img/07-try/qa/NGA_M83_582_F5_19_158-xd-int_2color_medium_annot.jpg
GALEX,GALEX,nga_m83_west-xd-int_2color_medium_annot.jpg,203.941758453579,-29.9696555012039,538319,2,[3840 3840],[0.000416667 0.000416667],image/jpeg,ICRS,TAN,[1920.5 1920.5],[79.6701 -66.0966],[-0.000416667 -0.0 -0.0 0.000416667],UV,meters,2.35e-07,3.007e-07,1.693e-07,http://galex.stsci.edu/data/GR6/pipe/01-vsn/05619-NGA_M83_west/d/01-main/0001-img/07-try/qa/NGA_M83_west-xd-int_2color_medium_annot.jpg
GALEX,GALEX,nga_m83_west-xd-int_2color.jpg,203.941758453579,-29.9696555012039,2559859,2,[3840 3840],[0.000416667 0.000416667],image/jpeg,ICRS,TAN,[1920.5 1920.5],[79.6701 -66.0966],[-0.000416667 -0.0 -0.0 0.000416667],UV,meters,2.35e-07,3.007e-07,1.693e-07,http://galex.stsci.edu/data/GR6/pipe/01-vsn/05619-NGA_M83_west/d/01-main/0001-img/07-try/qa/NGA_M83_west-xd-int_2color.jpg
GALEX,GALEX,nga_m83_west-xd-int_2color_large.jpg,203.941758453579,-29.9696555012039,477034,2,[3840 3840],[0.000416667 0.000416667],image/jpeg,ICRS,TAN,[1920.5 1920.5],[79.6701 -66.0966],[-0.000416667 -0.0 -0.0 0.000416667],UV,meters,2.35e-07,3.007e-07,1.693e-07,http://galex.stsci.edu/data/GR6/pipe/01-vsn/05619-NGA_M83_west/d/01-main/0001-img/07-try/qa/NGA_M83_west-xd-int_2color_large.jp

#### Download an image
For the first result, print the file format and download the file. If repeatedly executing this code, add `cache=True` to `download_file()` to prevent repeated downloads.

See `download_file()` documentation here: https://docs.astropy.org/en/stable/api/astropy.utils.data.download_file.html#astropy.utils.data.download_file

In [9]:
print(results[0].format)
file_name = download_file(results[0].getdataurl())  
file_name

image/fits


'/var/folders/t6/zv_75s3156780r5wyknqyb8w0000gn/T/astropy-download-35063-ow4awngb'

## 2.4 Spectral search
Example:  Find a spectral service for x-ray data.  Query it around Delta Ori with a search **diameter** of 10 arc minutes, and download the first data product.  Note that the results table can be inspected for potentially useful columns.

Spectral search is very similar to image search. In this example, note:
* **`diameter`** defines the size of the search region
* `waveband` used in `regsearch()`
* Astropy `Angle` used to specify radius units other than degrees.

In [10]:
# Search for a spectrum search service that has x-ray data.
services = vo.regsearch(servicetype='spectrum', waveband='x-ray')

# Assuming there are services and the first one is OK...
results = services[0].search(pos=SkyCoord.from_name("Delta Ori"), 
                             diameter=Angle(10 * u.arcmin))

# Assuming there are results, download the first file.
print(f'Title: {results[0].title}, Format: {results[0].format}')
file_name = download_file(results[0].getdataurl())  
file_name

Title: acisf00639N004_pha2, Format: application/fits


'/var/folders/t6/zv_75s3156780r5wyknqyb8w0000gn/T/astropy-download-35063-7fwo82uo'

## 2.5 Table search
Example:  Find the HEASARC Table Access Protocol (TAP) service, get some information about the available tables.

In [11]:
services = vo.regsearch(servicetype='tap', keywords=['heasarc'])
print(f'{len(services)} service(s) found.')
# We found only one service.  Print some info about the service and its tables.
print(f'{services[0].describe()}')
tables = services[0].service.tables  # Queries for details of the service's tables
print(f'{len(tables)} tables:')
for t in tables:
    print(f'{t.name:30s} - {t.description}')  # A more succinct option than t.describe()

1 service(s) found.
Table Access Protocol Service
HEASARC Xamin Catalog Interface
Short Name: HEASARC
IVOA Identifier: ivo://nasa.heasarc/services/xamin
Base URL: https://heasarc.gsfc.nasa.gov/xamin/vo/tap

The HEASARC is NASA domain archive for high-energy and microwave astronomy.
The Xamin interface provides access to over 600 observation and object tables.
This includes observation tables for more than 30 missions and observatories
and hundreds of derived object tables. Non-high energy tables are included to
make it easier for users to compare information.

Subjects: HEASARC
Waveband Coverage:
None


999 tables:
TAP_SCHEMA.columns             - Available columns
TAP_SCHEMA.key_columns         - Key/Foreign key pairs -- usused
TAP_SCHEMA.keys                - Joins between tables -- unused
TAP_SCHEMA.schemas             - Available schemas
TAP_SCHEMA.tables              - Available tables
"first"                        - Faint Images of the Radio Sky at Twenty cm (FIRST)
a1                             - HEAO 1 A1 X-Ray Source Catalog
a1point                        - HEAO 1 A1 Lightcurves
a2lcpoint                      - HEAO 1 A2 Pointed Lightcurves
a2lcscan                       - HEAO 1 A2 Scanned Lightcurves
a2led                          - HEAO 1 A2 LED Catalog
a2pic                          - HEAO 1 A2 Piccinotti Catalog
a2point                        - HEAO 1 A2 Pointing Catalog
a2rtraw                        - HEAO 1 A2 Raw Rates
a2specback                     - HEAO 1 A2 Spectra Background
a2spectra                      - HEAO 1 A2 Spectra
a3                             - HE


ngc2264cx2                     - NGC 2264 Chandra X-Ray Point Source Catalog 2
ngc2264cxo                     - NGC 2264 Chandra X-Ray Point Source Catalog
ngc2264xmm                     - NGC 2264 XMM-Newton X-Ray Point Source Catalog
ngc2362cxo                     - NGC 2362 Chandra X-Ray Point Source Catalog
ngc2403cx2                     - NGC 2403 Chandra X-Ray Point Source Catalog
ngc2403cxo                     - NGC 2403 Central 3-kpc Region Chandra Source Catalog
ngc2516cxo                     - NGC 2516 Chandra X-Ray Point Source Catalog
ngc2516xmm                     - NGC 2516 Cluster XMM-Newton X-Ray Point Source Catalog
ngc253xmm                      - NGC 253 XMM-Newton X-Ray Point Source Catalog
ngc2547xmm                     - NGC 2547 XMM-Newton X-Ray Point Source Catalog
ngc2808cxo                     - NGC 2808 Chandra X-Ray Point Source Catalog
ngc2808xmm                     - NGC 2808 XMM-Newton X-Ray Point Source Catalog
ngc2903cxo                     - NGC 2903 


sdsss82cxo                     - Sloan Digital Sky Survey Stripe 82 Chandra Source Match Catalog
sdsss82xmm                     - Sloan Digital Sky Survey Stripe 82 XMM-Newton Source Match Catal
sdssunuqsr                     - Sloan Digital Sky Survey Unusual Quasars Catalog
sdsswdsd                       - Sloan Digital Sky Survey DR4 White Dwarf & Hot Subdwarf Catalog
sdsswhlgc                      - Sloan Digital Sky Survey DR6 Galaxy Clusters Catalog
sdssxmmqso                     - Sloan Digital Sky Survey (DR5)/XMM-Newton Quasar Survey Catalog
selhcgcxo                      - Selected Hickson Compact Groups Chandra X-Ray Point Source Catal
sfgalhmxb                      - Star-Forming Galaxies High-Mass X-Ray Binaries Catalog
sfincspcm                      - Star Formation in Nearby Clouds (SFiNCs) Probable Cluster Member
sfincsxray                     - Star Formation in Nearby Clouds (SFiNCs) X-Ray Source Catalog
sgraregcsc                     - Sgr A* Region Compact Radio So


xmmslewegs                     - XMM-Newton Slew Survey Extragalactic Sample
xmmslewful                     - XMM-Newton Slew Survey Full Source Catalog, v2.0
xmmssc                         - XMM-Newton Serendipitous Source Catalog (4XMM-DR9 Version)
xmmsscgps                      - XMM-Newton Survey Science Center Survey of the Galactic Plane
xmmssclwbd                     - XMM-Newton 2XMMi-DR3 Selected Source Detections Catalog
xmmssclwbs                     - XMM-Newton 2XMMi-DR3 Selected Source Classifications Catalog
xmmstack                       - XMM-Newton Serendipitous Source Catalog from Stacked Observation
xmmstackob                     - XMM-Newton Serendipitous Source Catalog from Stacked Observation
xmmt2flare                     - 2XMM Flares Detected from Tycho-2 Stars
xmmvaragn                      - Ensemble X-Ray Variability of AGN in 2XMMi-DR3
xmmxassist                     - XMM-Newton XAssist Source List
xms                            - XMM-Newton Medium Sensit

#### Column Information
For any table, we can list the column names and descriptions.

In [12]:
for c in tables['zcat'].columns:
    print(f'{c.name:30s} - {c.description}')

"__z_ra_dec"                   - System unit vector column
ref_bmag                       - Reference Code for B Magnitude
"__y_ra_dec"                   - System unit vector column
comments                       - Comments
luminosity_class               - Luminosity Class
ref_redshift                   - Reference Code for Redshift
ugc_or_eso                     - UGC or ESO Number
class                          - Browse Object Classification
redshift                       - Redshift
diameter_1                     - Diameter 1 (arcmin)
bii                            - Galactic Latitude
"__x_ra_dec"                   - System unit vector column
"__row"                        - System index column
radial_velocity                - Heliocentric Radial Velocity (km/s)
morph_type                     - Morphological (T) Type
bt_mag                         - B_T Magnitude
rfn_number                     - Plate Number
name                           - Catalog Designation
notes                  

#### Perform a Query
Example:  Perform a cone search on the ZCAT catalog at M83 with a 1.0 degree radius.

In [13]:
coord = SkyCoord.from_name("m83")
query = f'''
SELECT ra, dec, Radial_Velocity, radial_velocity_error, bmag, morph_type FROM public.zcat as cat where 
contains(point('ICRS',cat.ra,cat.dec),circle('ICRS',{coord.ra.deg},{coord.dec.deg},1.0))=1
'''
results = services[0].service.run_async(query)

results.to_table()

ra,dec,radial_velocity,radial_velocity_error,bmag,morph_type
float64,float64,int32,int16,float32,int16
204.793,-30.7432,4795,--,16.7,--
203.9684,-30.4318,3847,53,--,--
203.5961,-30.3989,10310,300,--,--
203.9574,-30.3824,4196,--,16.4,--
203.7846,-30.1177,300,--,16.7,--
204.3646,-30.0394,14985,--,18.0,--
203.2306,-29.9193,300,--,16.6,--
204.2552,-29.8712,518,5,8.22,5
203.854,-29.8387,76746,300,--,--


# 3. Astroquery 
Many archives have Astroquery modules for data access, including:

* [HEASARC Queries (astroquery.heasarc)](https://astroquery.readthedocs.io/en/latest/heasarc/heasarc.html)
* [HITRAN Queries (astroquery.hitran)](https://astroquery.readthedocs.io/en/latest/hitran/hitran.html)
* [IRSA Image Server program interface (IBE) Queries (astroquery.ibe)](https://astroquery.readthedocs.io/en/latest/ibe/ibe.html)
* [IRSA Queries (astroquery.irsa)](https://astroquery.readthedocs.io/en/latest/irsa/irsa.html)
* [IRSA Dust Extinction Service Queries (astroquery.irsa_dust)](https://astroquery.readthedocs.io/en/latest/irsa/irsa_dust.html)
* [JPL Spectroscopy Queries (astroquery.jplspec)](https://astroquery.readthedocs.io/en/latest/jplspec/jplspec.html)
* [MAST Queries (astroquery.mast)](https://astroquery.readthedocs.io/en/latest/mast/mast.html)
* [NASA ADS Queries (astroquery.nasa_ads)](https://astroquery.readthedocs.io/en/latest/nasa_ads/nasa_ads.html)
* [NED Queries (astroquery.ned)](https://astroquery.readthedocs.io/en/latest/ned/ned.html)

For more, see https://astroquery.readthedocs.io/en/latest/

## 3.1 NED
Example:  Get an Astropy Table containing the objects from paper 2018ApJ...858...62K.  For more on the API, see https://astroquery.readthedocs.io/en/latest/ned/ned.html 

In [14]:
from astroquery.ned import Ned
objects_in_paper = Ned.query_refcode('2018ApJ...858...62K')
objects_in_paper

No.,Object Name,RA,DEC,Type,Velocity,Redshift,Redshift Flag,Magnitude and Filter,Separation,References,Notes,Photometry Points,Positions,Redshift Points,Diameter Points,Associations
,,degrees,degrees,,km / s,,,,arcmin,,,,,,,
int32,str30,float64,float64,object,float64,float64,object,object,float64,int32,int32,int32,int32,int32,int32,int32
1,WISEA J115844.06+273505.3,179.68371,27.585,G,455.0,0.001518,,17.2g,--,25,0,22,10,7,0,0
2,SDSS J121731.98+323157.7,184.38329,32.53271,G,447.0,0.001492,SPEC,17.95,--,5,0,15,4,2,4,0
3,SDSS J121811.04+465501.2,184.54614,46.91685,G,472.0,0.001576,,17.75,--,16,0,37,13,8,4,0
4,NGC 4455,187.18381,22.82045,G,637.0,0.002125,,12.93,--,193,5,68,28,21,11,0
5,Virgo Cluster,187.69708,12.33694,GClstr,1079.0,0.0036,,10.3J,--,1168,7,3,8,5,0,0
6,UGC 07678,188.00071,39.83267,G,666.0,0.002222,,14.3g,--,58,0,42,17,13,11,0
7,NGC 4618,190.38688,41.15078,G,544.0,0.001815,,11.22,--,334,12,113,41,20,8,1
8,SDSS J124354.70+412724.9,190.98208,41.45694,G,402.0,0.001341,,17.1g,--,8,0,17,8,3,3,0
